In [1]:
import polars as pl 
import numpy as np
from sklearn.datasets import make_classification
from sklearn.feature_selection import f_regression
import pandas as pd
from time import perf_counter
import sys
sys.path.append('../src')
from eda.eda_selection import _f_score, mrmr, MRMR_STRATEGY

In [2]:
# create some data
orig_x, orig_y = make_classification(n_samples = 500_000, n_features = 200, n_informative = 20, n_redundant = 180)

In [3]:
df_pl = pl.from_numpy(orig_x).insert_at_idx(0, pl.Series("target", orig_y))
df_pd = df_pl.to_pandas()

In [4]:
def mrmr_my_rewrite(df:pl.DataFrame, target:str, k:int) -> list[str]:

    features = df.columns
    features.remove(target)

    start = perf_counter()
    f_scores = _f_score(df, target, features)
    end_1 = perf_counter()
    print(f"Spent {end_1 - start:.2f}s to compute f score.")

    df_scaled = df.select(features).with_columns(
        (pl.col(f) - pl.col(f).mean())/pl.col(f).std() for f in features
    )

    cumulating_sum = np.zeros(len(features)) # For each feature at index i, we keep a cumulating sum
    top_idx = np.argmax(f_scores)
    selected_features = [features[top_idx]]
    for j in range(1, k): 
        argmax = -1
        current_max = -1
        last_selected = selected_features[-1]
        for i,f in enumerate(features):
            if f not in selected_features:
                # Left = cumulating sum of abs corr
                # Right = abs correlation btw last_selected and f
                cumulating_sum[i] += np.abs((df_scaled.get_column(last_selected)*df_scaled.get_column(f)).mean())
                denominator = cumulating_sum[i] / j
                new_score = f_scores[i] / denominator
                if new_score > current_max:
                    current_max = new_score
                    argmax = i

        selected_features.append(features[argmax])

    end_2 = perf_counter()
    print(f"Took {end_2 - end_1:.2f}s to compute mrmr.")
    return selected_features
    

In [5]:
def mrmr_np_rewrite(df:pl.DataFrame, target:str, k:int) -> list[str]:

    features = df.columns
    features.remove(target)

    start = perf_counter()
    f_scores = _f_score(df, target, features)
    end_1 = perf_counter()
    print(f"Spent {end_1 - start:.2f}s to compute f score.")

    x = df.select(features).to_numpy()
    x_scaled = (x - x.mean(axis=0)) / x.std(axis=0)

    cumulating_sum = np.zeros(len(features)) # For each feature at index i, we keep a cumulating sum
    top_idx = np.argmax(f_scores)
    selected_features = [features[top_idx]]
    last_selected_idx = top_idx
    for j in range(1, k): 
        argmax = -1
        current_max = -1
        for i,f in enumerate(features):
            if f not in selected_features:
                # Left = cumulating sum of abs corr
                # Right = abs correlation btw last_selected and f
                cumulating_sum[i] += np.abs(np.mean(x_scaled[:,last_selected_idx]*x_scaled[:,i]))
                denominator = cumulating_sum[i] / j
                new_score = f_scores[i] / denominator
                if new_score > current_max:
                    current_max = new_score
                    argmax = i

        selected_features.append(features[argmax])
        last_selected_idx = argmax

    end_2 = perf_counter()
    print(f"Took {end_2 - end_1:.2f}s to compute mrmr.")
    return selected_features

In [6]:
def mrmr_medium(df:pd.DataFrame, target:str, k:int) -> list[str]:

    features = list(df.columns)
    features.remove(target)

    X = df[features]
    y = df[target]

    # compute F-statistics and initialize correlation matrix
    start = perf_counter()
    F = pd.Series(f_regression(X, y)[0], index = X.columns)
    end_1 = perf_counter()
    print(f"Spent {end_1 - start:.2f}s to compute f score.")

    corr = pd.DataFrame(.00001, index = X.columns, columns = X.columns)

    # initialize list of selected features and list of excluded features
    selected = []
    not_selected = X.columns.to_list()

    # repeat K times
    for i in range(k):
    
        # compute (absolute) correlations between the last selected feature and all the (currently) excluded features
        if i > 0:
            last_selected = selected[-1]
            corr.loc[not_selected, last_selected] = X[not_selected].corrwith(X[last_selected]).abs().clip(.00001)
            
        # compute FCQ score for all the (currently) excluded features (this is Formula 2)
        score = F.loc[not_selected] / corr.loc[not_selected, selected].mean(axis = 1).fillna(.00001)
        
        # find best feature, add it to selected and remove it from not_selected
        best = score.index[score.argmax()]
        selected.append(best)
        not_selected.remove(best)

    # print(selected) 
    end_2 = perf_counter()
    print(f"Took {end_2 - end_1:.2f}s to compute mrmr.")
    return selected 

In [7]:
mrmr_my_rewrite(df_pl, "target", 50)

Spent 0.11s to compute f score.
Took 3.14s to compute mrmr.


['column_12',
 'column_166',
 'column_171',
 'column_78',
 'column_72',
 'column_152',
 'column_189',
 'column_117',
 'column_160',
 'column_7',
 'column_141',
 'column_101',
 'column_41',
 'column_199',
 'column_91',
 'column_125',
 'column_154',
 'column_8',
 'column_163',
 'column_197',
 'column_51',
 'column_29',
 'column_62',
 'column_196',
 'column_93',
 'column_165',
 'column_116',
 'column_159',
 'column_23',
 'column_40',
 'column_73',
 'column_64',
 'column_149',
 'column_114',
 'column_183',
 'column_26',
 'column_155',
 'column_19',
 'column_105',
 'column_3',
 'column_143',
 'column_16',
 'column_48',
 'column_128',
 'column_148',
 'column_140',
 'column_161',
 'column_157',
 'column_121',
 'column_168']

In [8]:
# Use my version of f_score computation.
# Use pure NumPy for the rest of the computation.
# This shows that copying to NumPy, does not pay off. Stay in Polars is a better idea.
mrmr_np_rewrite(df_pl, "target", 50)

Spent 0.12s to compute f score.
Took 14.16s to compute mrmr.


['column_12',
 'column_166',
 'column_171',
 'column_78',
 'column_72',
 'column_152',
 'column_189',
 'column_117',
 'column_160',
 'column_7',
 'column_141',
 'column_101',
 'column_41',
 'column_199',
 'column_91',
 'column_125',
 'column_154',
 'column_8',
 'column_163',
 'column_197',
 'column_51',
 'column_29',
 'column_62',
 'column_196',
 'column_93',
 'column_165',
 'column_116',
 'column_159',
 'column_23',
 'column_40',
 'column_73',
 'column_64',
 'column_149',
 'column_114',
 'column_183',
 'column_26',
 'column_155',
 'column_19',
 'column_105',
 'column_3',
 'column_143',
 'column_16',
 'column_48',
 'column_128',
 'column_148',
 'column_140',
 'column_161',
 'column_157',
 'column_121',
 'column_168']

In [9]:
mrmr_medium(df_pd, "target", 50)

Spent 0.28s to compute f score.
Took 59.35s to compute mrmr.


['column_12',
 'column_166',
 'column_171',
 'column_78',
 'column_72',
 'column_152',
 'column_189',
 'column_117',
 'column_160',
 'column_7',
 'column_141',
 'column_101',
 'column_41',
 'column_199',
 'column_91',
 'column_125',
 'column_154',
 'column_8',
 'column_163',
 'column_197',
 'column_51',
 'column_29',
 'column_62',
 'column_196',
 'column_93',
 'column_165',
 'column_116',
 'column_159',
 'column_23',
 'column_40',
 'column_73',
 'column_64',
 'column_149',
 'column_114',
 'column_183',
 'column_26',
 'column_155',
 'column_19',
 'column_105',
 'column_3',
 'column_143',
 'column_16',
 'column_48',
 'column_128',
 'column_148',
 'column_140',
 'column_161',
 'column_157',
 'column_121',
 'column_168']

In [10]:
# My yet to be published package. 2.8s
mrmr(df_pl, target="target", k = 50)

Top 5 feature importance is (by MRMR_STRATEGY.F_SCORE):
shape: (5, 2)
┌────────────┬───────────────────────┐
│ feature    ┆ MRMR_STRATEGY.F_SCORE │
│ ---        ┆ ---                   │
│ str        ┆ f64                   │
╞════════════╪═══════════════════════╡
│ column_12  ┆ 294005.656068         │
│ column_171 ┆ 211587.59218          │
│ column_78  ┆ 199253.929747         │
│ column_189 ┆ 143336.684677         │
│ column_117 ┆ 121793.034639         │
└────────────┴───────────────────────┘
Selected column_12 by MRMR. 1/50
Selected column_166 by MRMR. 2/50
Selected column_171 by MRMR. 3/50
Selected column_78 by MRMR. 4/50
Selected column_72 by MRMR. 5/50
Selected column_152 by MRMR. 6/50
Selected column_189 by MRMR. 7/50
Selected column_117 by MRMR. 8/50
Selected column_160 by MRMR. 9/50
Selected column_7 by MRMR. 10/50
Selected column_141 by MRMR. 11/50
Selected column_101 by MRMR. 12/50
Selected column_41 by MRMR. 13/50
Selected column_199 by MRMR. 14/50
Selected column_91 by MRMR

mrmr_rank,feature
i64,str
1,"""column_12"""
2,"""column_166"""
3,"""column_171"""
4,"""column_78"""
5,"""column_72"""
6,"""column_152"""
7,"""column_189"""
8,"""column_117"""
9,"""column_160"""


In [11]:
from mrmr import mrmr_classif

In [12]:
#
def mrmr_package(df:pd.DataFrame, target:str, k:int) -> list[str]:
    
    features = list(df.columns)
    features.remove(target)

    X = df[features]
    y = df[target]
    start = perf_counter()
    output = mrmr_classif(X, y, K = k)
    end = perf_counter()
    print(f"Spent {end - start:.2f}s to compute mrmr.")
    return output

In [13]:
mrmr_package(df_pd, "target", 50)

100%|██████████| 50/50 [01:01<00:00,  1.22s/it]

Spent 65.82s to compute mrmr.


['column_12',
 'column_166',
 'column_171',
 'column_78',
 'column_72',
 'column_152',
 'column_189',
 'column_117',
 'column_160',
 'column_7',
 'column_141',
 'column_101',
 'column_41',
 'column_199',
 'column_91',
 'column_125',
 'column_154',
 'column_8',
 'column_163',
 'column_197',
 'column_51',
 'column_29',
 'column_62',
 'column_196',
 'column_93',
 'column_165',
 'column_116',
 'column_159',
 'column_23',
 'column_40',
 'column_73',
 'column_64',
 'column_149',
 'column_114',
 'column_183',
 'column_26',
 'column_155',
 'column_19',
 'column_105',
 'column_3',
 'column_143',
 'column_16',
 'column_48',
 'column_128',
 'column_148',
 'column_140',
 'column_161',
 'column_157',
 'column_121',
 'column_168']

In [15]:
mrmr(df_pl, target="target", k=20, strategy=MRMR_STRATEGY.RF, params={"n_estimators":20, "max_depth":5, "n_jobs":-1})

Random forest is not deterministic by default. Results may vary.
Top 5 feature importance is (by MRMR_STRATEGY.RF):
shape: (5, 2)
┌────────────┬──────────────────┐
│ feature    ┆ MRMR_STRATEGY.RF │
│ ---        ┆ ---              │
│ str        ┆ f64              │
╞════════════╪══════════════════╡
│ column_12  ┆ 0.095529         │
│ column_171 ┆ 0.067189         │
│ column_189 ┆ 0.060785         │
│ column_117 ┆ 0.058921         │
│ column_197 ┆ 0.058851         │
└────────────┴──────────────────┘
Selected column_12 by MRMR. 1/20
Selected column_6 by MRMR. 2/20
Selected column_193 by MRMR. 3/20
Selected column_117 by MRMR. 4/20
Selected column_197 by MRMR. 5/20
Selected column_7 by MRMR. 6/20
Selected column_171 by MRMR. 7/20
Selected column_141 by MRMR. 8/20
Selected column_189 by MRMR. 9/20
Selected column_154 by MRMR. 10/20
Selected column_78 by MRMR. 11/20
Selected column_114 by MRMR. 12/20
Selected column_137 by MRMR. 13/20
Selected column_101 by MRMR. 14/20
Selected column_41 by

mrmr_rank,feature
i64,str
1,"""column_12"""
2,"""column_6"""
3,"""column_193"""
4,"""column_117"""
5,"""column_197"""
6,"""column_7"""
7,"""column_171"""
8,"""column_141"""
9,"""column_189"""


In [16]:
mrmr(df_pl, target="target", k=20, strategy=MRMR_STRATEGY.XGB, params={"n_estimators":20, "max_depth":5})

XGB is not deterministic by default. Results may vary.
Top 5 feature importance is (by MRMR_STRATEGY.XGB):
shape: (5, 2)
┌────────────┬───────────────────┐
│ feature    ┆ MRMR_STRATEGY.XGB │
│ ---        ┆ ---               │
│ str        ┆ f64               │
╞════════════╪═══════════════════╡
│ column_12  ┆ 0.172563          │
│ column_101 ┆ 0.069675          │
│ column_171 ┆ 0.030006          │
│ column_117 ┆ 0.0298            │
│ column_8   ┆ 0.024688          │
└────────────┴───────────────────┘
Selected column_12 by MRMR. 1/20
Selected column_86 by MRMR. 2/20
Selected column_6 by MRMR. 3/20
Selected column_101 by MRMR. 4/20
Selected column_184 by MRMR. 5/20
Selected column_87 by MRMR. 6/20
Selected column_93 by MRMR. 7/20
Selected column_117 by MRMR. 8/20
Selected column_7 by MRMR. 9/20
Selected column_75 by MRMR. 10/20
Selected column_8 by MRMR. 11/20
Selected column_171 by MRMR. 12/20
Selected column_154 by MRMR. 13/20
Selected column_10 by MRMR. 14/20
Selected column_83 by MRM

mrmr_rank,feature
i64,str
1,"""column_12"""
2,"""column_86"""
3,"""column_6"""
4,"""column_101"""
5,"""column_184"""
6,"""column_87"""
7,"""column_93"""
8,"""column_117"""
9,"""column_7"""
